In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt

# for loading images
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.applications.xception import decode_predictions

%matplotlib inline

2024-12-07 10:39:07.400102: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# fixed seed
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [3]:
def make_model():
    ###################################
    ### architecture here(flow of data
    # process input nad assign to base
    inputs = keras.Input(shape=(200, 200, 3))

    # conv layer
    conv = keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(inputs)

    # pooling
    vectors = keras.layers.MaxPooling2D((2,2))(conv)

    # flatten vectors
    flattened_vectors = keras.layers.Flatten()(vectors)
    
    # inner layer that takes the vectors
    inner = keras.layers.Dense(64, activation='relu')(flattened_vectors)

    # another dense layer that will produce the output
    outputs = keras.layers.Dense(1, activation='sigmoid')(inner)
    
    # # add a dropout layer and pass the inner layer's output to it.
    # drop = keras.layers.Dropout(droprate)(inner)
    
    model = keras.Model(inputs, outputs)

    ###################################
    
    optimizer = keras.optimizers.SGD(learning_rate=0.002, momentum=0.8)
    loss = keras.losses.BinaryCrossentropy()

    model.compile(optimizer=optimizer, 
              loss=loss, 
              metrics=['accuracy'])
    
    return model

# Question 1
### Since we have a binary classification problem, what is the best loss function for us?

Answer: ***Binary Cross Entropy***

# Question 2

In [4]:
model = make_model()

2024-12-07 10:39:08.748594: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-12-07 10:39:08.753375: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-12-07 10:39:08.753533: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [5]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 200, 200, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 198, 198, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 99, 99, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 313632)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │    20,072,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,073,473 (76.57 MB)

 Trainable params: 20,073,473 (76.57 MB)

 Non-trainable params: 0 (0.00 B)

# Question 3

In [17]:
train_gen = ImageDataGenerator(rescale=1./255)
train_ds = train_gen.flow_from_directory(
    'data/train', 
    target_size=(200, 200), 
    batch_size=20,
    class_mode='binary'
)

val_gen = ImageDataGenerator(rescale=1./255)
val_ds = val_gen.flow_from_directory(
    'data/test', 
    target_size=(200, 200),
    batch_size=20,
    class_mode='binary'
)

Found 800 images belonging to 2 classes.
Found 201 images belonging to 2 classes.


In [18]:
model = make_model()
history = model.fit(
    train_ds,
    epochs=10,
    validation_data=val_ds
)

Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 74ms/step - accuracy: 0.5103 - loss: 0.7471 - val_accuracy: 0.5970 - val_loss: 0.6433
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 66ms/step - accuracy: 0.6542 - loss: 0.6163 - val_accuracy: 0.6119 - val_loss: 0.6770
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 65ms/step - accuracy: 0.6647 - loss: 0.6053 - val_accuracy: 0.6418 - val_loss: 0.6145
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 67ms/step - accuracy: 0.6944 - loss: 0.5762 - val_accuracy: 0.6468 - val_loss: 0.6081
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 66ms/step - accuracy: 0.7125 - loss: 0.5567 - val_accuracy: 0.6517 - val_loss: 0.6157
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 63ms/step - accuracy: 0.7175 - loss: 0.5353 - val_accuracy: 0.6866 - val_loss: 0.6124
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 67ms/step - accuracy: 0.7731 - loss: 0.4701 - val_accuracy: 0.6667 - val_loss: 0.6100
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 66ms/step - accuracy: 0.7696 - loss: 0.4701 - val_accuracy: 0.6269 - v

In [19]:
np.median(history.history['accuracy'])

0.7356249988079071

# Question 4

In [20]:
np.std(history.history['loss'])

0.08465469350413153

# Question 5

In [10]:
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=50,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_ds = train_gen.flow_from_directory(
    'data/train', 
    target_size=(200, 200), 
    batch_size=20,
    class_mode='binary'
)

val_gen = ImageDataGenerator(
    rescale=1./255,
)

val_ds = val_gen.flow_from_directory(
    'data/test', 
    target_size=(200, 200), 
    batch_size=20, 
    class_mode='binary'
)

Found 800 images belonging to 2 classes.
Found 201 images belonging to 2 classes.


In [11]:
# retrain model with 10 more epochs without creating a new one as instructed
history = model.fit(
    train_ds,
    epochs=10,
    validation_data=val_ds
)

Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - accuracy: 0.6988 - loss: 0.5703

/home/kabs/miniconda3/envs/tf/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


40/40 ━━━━━━━━━━━━━━━━━━━━ 6s 149ms/step - accuracy: 0.6980 - loss: 0.5713 - val_accuracy: 0.7065 - val_loss: 0.5847
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 6s 146ms/step - accuracy: 0.6165 - loss: 0.6204 - val_accuracy: 0.7214 - val_loss: 0.5561
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 6s 141ms/step - accuracy: 0.7016 - loss: 0.5775 - val_accuracy: 0.6915 - val_loss: 0.5620
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 6s 143ms/step - accuracy: 0.7049 - loss: 0.5648 - val_accuracy: 0.7214 - val_loss: 0.5567
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 6s 141ms/step - accuracy: 0.7032 - loss: 0.5751 - val_accuracy: 0.7164 - val_loss: 0.5725
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 6s 144ms/step - accuracy: 0.7081 - loss: 0.5446 - val_accuracy: 0.7363 - val_loss: 0.5668
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 6s 141ms/step - accuracy: 0.7014 - loss: 0.5668 - val_accuracy: 0.7264 - val_loss: 0.5337
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 6s 143ms/step - accuracy: 0.7439 - loss: 0.5322 - val_accuracy: 0.7313 - val_

In [15]:
np.mean(history.history['val_loss']).round(3)

0.563

In [16]:
np.mean(history.history['val_accuracy'][-5:]).round(3)

0.725